In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import zipfile 
from shapely.geometry import Point
# extract the gtfs files
# unzip the gtfs files
bus_zipfile = "../lacmta/current-base/gtfs_bus.zip"
rail_zipfile = "../lacmta-rail/current/gtfs_rail.zip"

def extract_gtfs_files():
    print("Extracting GTFS files...")
    extract_start = timeit.default_timer()
    with zipfile.ZipFile(bus_zipfile, 'r') as zipObj:
        zipObj.extractall()
    with zipfile.ZipFile(rail_zipfile, 'r') as zipObj:
        zipObj.extractall()
    extract_end = timeit.default_timer()
    print("Extracting GTFS files took " + str(extract_end - extract_start) + " seconds.")

import os
import sys
from pathlib import Path

def extract_zip_file_to_temp_directory(agency_id):
    zip_file = None
    if agency_id == 'lacmta':
        zip_file = '../lacmta/current-base/gtfs_bus.zip'
    elif agency_id == 'lacmta-rail':
        zip_file = '../lacmta-rail/current/gtfs_rail.zip'
    try:
        print('Extracting zip file to temp directory: ' + zip_file)
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall('../temp/'+agency_id)
    except Exception as e:
        print('Error extracting zip file to temp directory: ' + str(e))
        sys.exit(1)

extract_zip_file_to_temp_directory('lacmta')
extract_zip_file_to_temp_directory('lacmta-rail')


Extracting zip file to temp directory: ../lacmta/current-base/gtfs_bus.zip
Extracting zip file to temp directory: ../lacmta-rail/current/gtfs_rail.zip


In [9]:
list_of_gtfs_static_files = ["routes", "trips", "stops", "calendar", "shapes","stop_times"]
debug = True


def update_stops_seperately(temp_df_bus,temp_df_rail,file):
    temp_df_bus['agency_id'] = 'LACMTA'
    temp_gdf_bus_stops = gpd.GeoDataFrame(temp_df_bus,geometry=gpd.points_from_xy(temp_df_bus.stop_lon, temp_df_bus.stop_lat))
    temp_gdf_bus_stops.set_crs(epsg=4326, inplace=True)
    temp_df_rail['agency_id'] = 'LACMTA_Rail'
    temp_gdf_bus_stops['stop_id'] = temp_gdf_bus_stops['stop_id'].astype('str')
    temp_gdf_bus_stops['stop_code'] = temp_gdf_bus_stops['stop_code'].astype('str')
    temp_gdf_bus_stops['parent_station'] = temp_gdf_bus_stops['parent_station'].astype('str')
    temp_gdf_bus_stops['tpis_name'] = temp_gdf_bus_stops['tpis_name'].astype('str')

    temp_gdf_rail_stops = gpd.GeoDataFrame(temp_df_rail,geometry=gpd.points_from_xy(temp_df_rail.stop_lon, temp_df_rail.stop_lat))
    temp_gdf_rail_stops.set_crs(epsg=4326, inplace=True)
    temp_gdf_rail_stops['stop_id'] = temp_gdf_rail_stops['stop_id'].astype('str')
    temp_gdf_rail_stops['stop_code'] = temp_gdf_rail_stops['stop_code'].astype('str')
    temp_gdf_rail_stops['parent_station'] = temp_gdf_rail_stops['parent_station'].astype('str')
    temp_gdf_rail_stops['tpis_name'] = temp_gdf_rail_stops['tpis_name'].astype('str')
    return pd.concat([temp_gdf_bus_stops,temp_gdf_rail_stops])

def create_gdf_for_shapes(temp_df_bus,temp_df_rail):
    temp_gdf_bus = gpd.GeoDataFrame(temp_df_bus, geometry=gpd.points_from_xy(temp_df_bus.shape_pt_lon, temp_df_bus.shape_pt_lat))   
    temp_gdf_rail = gpd.GeoDataFrame(temp_df_rail, geometry=gpd.points_from_xy(temp_df_rail.shape_pt_lon, temp_df_rail.shape_pt_lat))
    shapes_combined_gdf = gpd.GeoDataFrame(pd.concat([temp_gdf_bus, temp_gdf_rail],ignore_index=True),geometry='geometry')
    shapes_combined_gdf.crs = 'EPSG:4326'
    return shapes_combined_gdf
def get_stop_times_for_trip_id(this_row):
    this_trips_df = stop_times_df.loc[stop_times_df['trip_id'] == this_row.trip_id]
    this_trips_df['route_id'] = this_row.route_id
    this_trips_df['direction_id'] = this_row.direction_id
    this_trips_df['day_type'] = this_row.day_type
    this_trips_df['geojson'] = this_trips_df.apply(lambda x: get_stops_data_based_on_stop_id(x.stop_id),axis=1)
    this_trips_df['stop_name'] = this_trips_df.apply(lambda x: stops_df.loc[stops_df['stop_id'] == str(x.stop_id)]['stop_name'].values[0],axis=1)
    simplified_df = this_trips_df[['route_id','route_code','stop_id','day_type','stop_sequence','direction_id','stop_name','geojson','agency_id']]
    
    df_to_combine.append(simplified_df)
    return simplified_df
def encode_lat_lon_to_geojson(lat,lon):
    this_geojson = {
        "type":"Feature",
        "geometry":{
            "type":"Point",
            "coordinates": [lon,lat]
        }
    }
    return this_geojson

def get_stops_data_based_on_stop_id(stop_id):
    this_stops_df = stops_df.loc[stops_df['stop_id'] == str(stop_id)]
    new_object = encode_lat_lon_to_geojson(this_stops_df['stop_lat'].values[0],this_stops_df['stop_lon'].values[0])
    return new_object

df_to_combine = []

def create_list_of_trips(trips,stop_times):
    print('Creating list of trips')
    global trips_list_df
    trips_list_df = stop_times.groupby('trip_id')['stop_sequence'].max().sort_values(ascending=False).reset_index()
    trips_list_df = trips_list_df.merge(stop_times[['trip_id','stop_id','stop_sequence','route_code']], on=['trip_id','stop_sequence'])
    return trips_list_df



def get_stop_times_from_stop_id(this_row):
    trips_by_route_df = trips_df.loc[trips_df['route_id'] == this_row.route_id]
    stop_times_by_trip_df = stop_times_df[stop_times_df['trip_id'].isin(trips_by_route_df['trip_id'])]

    # get the stop times for this stop id
    this_stops_df = stop_times_by_trip_df.loc[stop_times_by_trip_df['stop_id'] == this_row.stop_id]
    this_stops_df = this_stops_df.sort_values(by=['departure_time'],ascending=True)
    departure_times_array = this_stops_df['departure_time'].values.tolist()
    return departure_times_array

def get_day_type_from_service_id(row):
    # print('Getting day type from service id')
    cleaned_row = str(row).lower()
    if 'weekday' in cleaned_row:
        return 'weekday'
    elif 'saturday' in cleaned_row:
        return 'saturday'
    elif 'sunday' in cleaned_row:
        return 'sunday'
def get_lat_long_from_coordinates(geojson):
    this_geojson_geom = geojson['geometry']
    return Point(this_geojson_geom['coordinates'][0], this_geojson_geom['coordinates'][1])    
def update_gtfs_static_files():
    global stop_times_df
    global trips_df
    global calendar_dates_df
    global calendar_df
    global stops_df
    for file in list_of_gtfs_static_files:
        print("******************")
        print("Starting with " + file)
        bus_file_path = ""
        rail_file_path = ""

        bus_file_path = "../temp/lacmta/" + file + '.txt'
        rail_file_path = "../temp/lacmta-rail/" + file + '.txt'
        temp_df_bus = pd.read_csv(bus_file_path)
        temp_df_bus['agency_id'] = 'LACMTA'
        temp_df_rail = pd.read_csv(rail_file_path)
        temp_df_rail['agency_id'] = 'LACMTA_Rail'
        if file == "stops":
            stops_df = update_stops_seperately(temp_df_bus,temp_df_rail,file)
        elif file == "shapes":
            shapes_combined_gdf = create_gdf_for_shapes(temp_df_bus,temp_df_rail)
        elif file == "stop_times":            
            print('temp_df_rail')
            print(temp_df_rail.head())
            temp_df_rail['trip_id'] = temp_df_rail['trip_id'].astype(str)
            temp_df_bus['trip_id'] = temp_df_bus['trip_id'].astype(str)
            cols = ['pickup_type','drop_off_type']
            combined_temp_df = pd.concat([temp_df_bus, temp_df_rail])
            combined_temp_df['rider_usage_code_before_coding'] = combined_temp_df[cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)
            combined_temp_df['rider_usage_code'] = combined_temp_df['rider_usage_code_before_coding'].apply(lambda x: 1 if x == '00' else 2 if x == '10' else 3 if x == '01' else 0 if x == '11' else -1)
            if 'bay_num' not in combined_temp_df.columns:
                combined_temp_df['bay_num'] = -1
            combined_temp_df.drop(columns=['rider_usage_code_before_coding'])
            stop_times_df = combined_temp_df

        else:
            combined_temp_df = pd.concat([temp_df_bus, temp_df_rail])

            if file == "trips":
                trips_df = combined_temp_df
            if file == "calendar_dates":
                calendar_dates_df = combined_temp_df
            if file == "calendar":
                calendar_df = combined_temp_df

                # combined_temp_df.to_sql(file,engine,index=False,if_exists="replace",schema=TARGET_SCHEMA)
    print("Processing trip list")
    trips_list_df = create_list_of_trips(trips_df,stop_times_df)
    summarized_trips_df = trips_df[["route_id","trip_id","direction_id","service_id","agency_id"]]
    summarized_trips_df['day_type'] = summarized_trips_df['service_id'].map(get_day_type_from_service_id)
    trips_list_df = trips_list_df.merge(summarized_trips_df, on='trip_id').drop_duplicates(subset=['route_id','day_type','direction_id'])
    trips_list_df.apply(lambda row: get_stop_times_for_trip_id(row), axis=1)
    stop_times_by_route_df = pd.concat(df_to_combine)
    stop_times_by_route_df['departure_times'] = stop_times_by_route_df.apply(lambda row: get_stop_times_from_stop_id(row),axis=1)
    stop_times_by_route_df['route_code'].fillna(stop_times_by_route_df['route_id'], inplace=True)
    route_stops_geo_data_frame = gpd.GeoDataFrame(stop_times_by_route_df, geometry=stop_times_by_route_df.apply(lambda x: get_lat_long_from_coordinates(x.geojson),axis=1))
    route_stops_geo_data_frame.set_crs(epsg=4326, inplace=True)
    # Perform the joins
    print("Processing trip_shape_stops_df ...")
    # Merge stop_times_df with trips_df on 'trip_id'
    merged_df = pd.merge(stop_times_df, trips_df, on='trip_id')

    # Create trip_shape_stops_df
    trip_shape_stops_df = merged_df.groupby(['trip_id', 'shape_id'])['stop_id'].apply(list).reset_index()
    print("Processing unique shape stop times...")
    

    # Perform the joins
    df = pd.merge(trip_shape_stops_df, trips_df, on=['trip_id', 'shape_id'])
    df = pd.merge(df, route_stops_geo_data_frame, on='route_id')
    df = pd.merge(df, shapes_combined_gdf, on='shape_id')

    # Get unique route_codes
    unique_route_codes = df['route_code'].unique()

    # Create an empty DataFrame to store the results
    result_df = pd.DataFrame()

    # Process each unique route_code
    for route_code in unique_route_codes:
        df_route_code = df[df['route_code'] == route_code]

        # Get unique direction_ids for the current route_code
        unique_direction_ids = df_route_code['direction_id'].unique()

        # Process each unique direction_id
        for direction_id in unique_direction_ids:
            df_route = df_route_code[df_route_code['direction_id'] == direction_id]

            # Sort the DataFrame
            df_route = df_route.sort_values(['service_id', 'trip_id', 'stop_sequence'])

            # Append the sorted DataFrame to the result DataFrame
            result_df = result_df.append(df_route)

    # Write the result DataFrame to a new table in the database
    print("Done processing unique shape stop times.")


update_gtfs_static_files()

******************
Starting with routes
******************
Starting with trips
******************
Starting with stops
******************
Starting with calendar
******************
Starting with shapes
******************
Starting with stop_times


C:\Users\etria\AppData\Local\Temp\ipykernel_9912\1134030024.py:102: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df_bus = pd.read_csv(bus_file_path)


temp_df_rail
    trip_id arrival_time departure_time  stop_id  stop_sequence  \
0  58503960     05:07:00       05:07:00    80101              1   
1  58503960     05:09:00       05:09:00    80102              2   
2  58503960     05:13:00       05:13:00    80105              3   
3  58503960     05:16:00       05:16:00    80106              4   
4  58503960     05:20:00       05:20:00    80107              5   

                                 stop_headsign  pickup_type  drop_off_type  \
0  Metro A-Line - APU / Citrus College Station            0              0   
1  Metro A-Line - APU / Citrus College Station            0              0   
2  Metro A-Line - APU / Citrus College Station            0              0   
3  Metro A-Line - APU / Citrus College Station            0              0   
4  Metro A-Line - APU / Citrus College Station            0              0   

     route_code              destination_code  timepoint    agency_id  
0  Metro A-Line  APU / Citrus College Stati

C:\Users\etria\AppData\Local\Temp\ipykernel_9912\1134030024.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summarized_trips_df['day_type'] = summarized_trips_df['service_id'].map(get_day_type_from_service_id)


Processing trip_shape_stops_df ...
Processing unique shape stop times...


MemoryError: Unable to allocate 56.1 GiB for an array with shape (7534677903,) and data type int64

In [ ]:
# Assuming stop_times_df, trips_df, route_stops_geo_data_frame, shapes_combined_gdf are already defined

# Select only the necessary columns from stop_times
stop_times_df = stop_times_df[['trip_id', 'stop_id', 'arrival_time']]

# Aggregate the data if necessary
# For example, get the first and last arrival time for each trip
stop_times_df = stop_times_df.groupby('trip_id').agg({'arrival_time': ['min', 'max']}).reset_index()

# Create trip_shape_stops_df
trip_shape_stops_df = merged_df.groupby(['trip_id', 'shape_id'])['stop_id'].apply(list).reset_index()
print("Processing unique shape stop times...")

# Perform the joins
df = pd.merge(trip_shape_stops_df, stop_times_df, on='trip_id')
df = pd.merge(df, trips_df, on='trip_id')
df = pd.merge(df, route_stops_geo_data_frame, on='route_id')
df = pd.merge(df, shapes_combined_gdf, on='shape_id')

# Get unique route_codes
unique_route_codes = df['route_code'].unique()

# Create an empty DataFrame to store the results
result_df = pd.DataFrame()

# Process each unique route_code
for route_code in unique_route_codes:
    df_route_code = df[df['route_code'] == route_code]

    # Get unique direction_ids for the current route_code
    unique_direction_ids = df_route_code['direction_id'].unique()

    # Process each unique direction_id
    for direction_id in unique_direction_ids:
        df_route = df_route_code[df_route_code['direction_id'] == direction_id]

        # Sort the DataFrame
        df_route = df_route.sort_values(['service_id', 'trip_id', 'stop_sequence'])

        # Append the sorted DataFrame to the result DataFrame
        result_df = result_df.append(df_route)

# Write the result DataFrame to a new table in the database
print("Done processing unique shape stop times.")

In [ ]:
# Perform the joins
df = pd.merge(trip_shape_stops_df, trips_df[['trip_id', 'direction_id', 'service_id']], on='trip_id')
df = pd.merge(df, stop_times_df[['trip_id', 'stop_id', 'departure_time']], on=['trip_id', 'stop_id'])

# Assuming calendar_df is your DataFrame containing 'service_id' and 'day_of_week'
df = pd.merge(df, calendar_df, on='service_id')

# Group by 'direction_id', 'day_of_week', 'trip_id', 'stop_id' and aggregate 'departure_time'
df = df.groupby(['direction_id', 'day_of_week', 'trip_id', 'stop_id'])['departure_time'].apply(list).reset_index()

# Pivot the DataFrame to get the desired hierarchy
df = df.pivot_table(index=['direction_id', 'day_of_week', 'trip_id', 'stop_id'], values='departure_time', aggfunc='first').reset_index()

# Convert the DataFrame to a dictionary for easier viewing of the hierarchy
data_dict = df.to_dict('records')

# Write the result DataFrame to a new table in the database
print("Done processing unique shape stop times.")

In [ ]:
# Merge stop_times_df with trips_df on 'trip_id'
merged_df = pd.merge(stop_times_df, trips_df, on='trip_id')

# Create trip_shape_stops_df
trip_shape_stops_df = merged_df.groupby(['trip_id', 'shape_id'])['stop_id'].apply(list).reset_index()

In [ ]:
# Merge stop_times_df with trips_df on 'trip_id'
merged_df = pd.merge(stop_times_df, trips_df, on='trip_id')

# Create trip_shape_stops_df
trip_shape_stops_df = merged_df.groupby(['trip_id', 'shape_id'])['stop_id'].apply(list).reset_index()

In [4]:
# Select only necessary columns
stop_times_df = stop_times_df[['trip_id', 'route_code', 'stop_sequence']]
trips_df = trips_df[['trip_id', 'route_id', 'shape_id']]

# Perform merge operation
df = pd.merge(stop_times_df, trips_df, on='trip_id')

# Continue with the rest of your code...
df = pd.merge(df, route_stops_geo_data_frame[['route_id']], on='route_id')
df = pd.merge(df, shapes_combined_gdf[['shape_id']], on='shape_id')

# Sort values before merge
df = df.sort_values(['trip_id', 'stop_sequence'])

# Use groupby instead of looping
grouped = df.groupby(['route_code'])

# Use concat instead of append
result_df = pd.concat([group.sort_values(['trip_id', 'stop_sequence']) for name, group in grouped])

NameError: name 'route_stops_geo_data_frame' is not defined

In [ ]:
# Check if 'direction_id' and 'service_id' exist in stop_times_df
if 'direction_id' in stop_times_df.columns and 'service_id' in stop_times_df.columns:
    df = pd.merge(stop_times_df[['trip_id', 'route_code', 'direction_id', 'service_id', 'stop_sequence']], 
                trips_df[['trip_id']], on='trip_id')
else:
    print("Columns 'direction_id' and/or 'service_id' do not exist in stop_times_df")
    df = pd.merge(stop_times_df[['trip_id', 'route_code', 'stop_sequence']], 
                trips_df[['trip_id']], on='trip_id')

df = pd.merge(df, route_stops_geo_data_frame[['route_id']], on='route_id')
df = pd.merge(df, shapes_combined_gdf[['shape_id']], on='shape_id')

# Sort values before merge
df = df.sort_values(['trip_id', 'stop_sequence'])

# Use groupby instead of looping
grouped = df.groupby(['route_code'])

# Use concat instead of append
result_df = pd.concat([group.sort_values(['trip_id', 'stop_sequence']) for name, group in grouped])

In [ ]:
# Merge only necessary columns
df = pd.merge(stop_times_df[['trip_id', 'route_code', 'direction_id', 'service_id', 'stop_sequence']], 
              trips_df[['trip_id']], on='trip_id')
df = pd.merge(df, route_stops_geo_data_frame[['route_id']], on='route_id')
df = pd.merge(df, shapes_combined_gdf[['shape_id']], on='shape_id')

# Sort values before merge
df = df.sort_values(['service_id', 'trip_id', 'stop_sequence'])

# Use groupby instead of looping
grouped = df.groupby(['route_code', 'direction_id'])

# Use concat instead of append
result_df = pd.concat([group.sort_values(['service_id', 'trip_id', 'stop_sequence']) for name, group in grouped])

In [ ]:
# Merge only necessary columns
df = pd.merge(stop_times_df[['trip_id', 'route_code', 'direction_id', 'service_id', 'stop_sequence']], 
              trips_df[['trip_id']], on='trip_id')
df = pd.merge(df, route_stops_geo_data_frame[['route_id']], on='route_id')
df = pd.merge(df, shapes_combined_gdf[['shape_id']], on='shape_id')

# Sort values before merge
df = df.sort_values(['service_id', 'trip_id', 'stop_sequence'])

# Use groupby instead of looping
grouped = df.groupby(['route_code', 'direction_id'])

# Use concat instead of append
result_df = pd.concat([group.sort_values(['service_id', 'trip_id', 'stop_sequence']) for name, group in grouped])